In [12]:
#import uproot
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np
import os,sys,glob,time
import csv

from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy.signal import find_peaks,peak_prominences
import scipy.optimize as opt
import matplotlib.mlab as mlab
from scipy.optimize import leastsq
#from sklearn import preprocessing
import peakutils

from astropy.stats import sigma_clipped_stats
from photutils.datasets import make_100gaussians_image
#from photutils import find_peaks
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
#from photutils import CircularAperture
import matplotlib.ticker as ticker
from matplotlib.ticker import AutoMinorLocator, MultipleLocator, FuncFormatter

#import memory_profiler

from IPython.display import Markdown as md


import re
numeric_const_pattern = '[-+]? (?: (?: \d* \. \d+ ) | (?: \d+ \.? ) )(?: [Ee] [+-]? \d+ ) ?'
rx = re.compile(numeric_const_pattern, re.VERBOSE)


#plt.style.use('seaborn-white')

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 22
plt.rcParams['axes.labelsize'] = 22
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 22
plt.rcParams['xtick.labelsize'] = 22
plt.rcParams['ytick.labelsize'] = 22
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.titlesize'] = 22


rootdir = "./data"
recondir = "./data"
file_name_conv="Config-verison-iso-nevts-seed#(_type).root"


In [13]:

def file_info_from_file(file):
    #fdata=file.find("data")
    #file=file[fdata:]
    
    try:
        first_dash=file.find("-")
        last_slash=file.rfind("/")
        Config=file[last_slash+1:first_dash]
        file=file[first_dash+1:]
    except:
        print("Issue with Config")
        exit()
    try:
        version="0"
        if file.find("v")>=0:
            version_dash=file.find("-")
            version=file[:version_dash]
            file=file[version_dash+1:]
        else:
            verison="0"
    except:
        print("issue with version")
        exit()
    try:
        tgt_dash=file.find("-")
        tgt=file[:tgt_dash]
        file=file[tgt_dash+1:]

        evts_dash=file.find("-")
        numevents=int(file[:evts_dash])
        seed=file[file.find("seed")+4:file.find(".")]
    except:
        print("issue with seed or numevents")
        return 0,0,0,0,0
        
    return Config,tgt,numevents,seed,version

def file_dict_from_file(file):
    
    fileinfo=file_info_from_file(file)
    
    fi=fileinfo
    if fileinfo[0] == 0:
        print("Double check file name")
        print("Recall: naming convention : %s "%file_name_conv)
        
        return False
    
    simFiledict={}
    simFiledict["Config"]=fileinfo[0]
    simFiledict["Target"]=fileinfo[1]
    simFiledict["NumEvents"]=fileinfo[2]
    simFiledict["Seed"]=fileinfo[3]
    simFiledict["Version"]=fileinfo[4]
    simFiledict["root_filename"]="%s/%s-%s-%s-%s-seed%s.root"%(rootdir,
                                                                                           fi[0],
                                                                                           fi[4],
                                                                                           fi[1],
                                                                                           fi[2],
                                                                                           fi[3])
    
    simFiledict["sens_filename"]="%s/%s-%s-%s-%s-seed%s_sensrecon.root"%(recondir,
                                                                                           fi[0],
                                                                                           fi[4],
                                                                                           fi[1],
                                                                                           fi[2],
                                                                                           fi[3])
    
    
    return simFiledict







In [10]:
def NexoDB():
    return pd.read_pickle("/home/wrkshp/nexo/datacheck/stdout_nexodatabase.pkl")


    

In [11]:
def BlankNexoDF_Sens():
    
    return pd.DataFrame(columns= ['energy', 'energy_mctruth_allLXe', 'energy_mctruth_inTPC',
       'r_max_simple', 'lower_z', 'upper_z', 'standoff', 'standoff_r',
       'standoff_z', 'passed_z_thresh', 'passed_xy_thresh', 'r_max_3d',
       'n_x_ch_abovenoise', 'n_y_ch_abovenoise', 'max_r_mctruth',
       'lower_z_mctruth', 'upper_z_mctruth', 'standoff_mctruth',
       'standoff_r_mctruth', 'standoff_z_mctruth', 'max_r_active_mctruth',
       'lower_z_active_mctruth', 'upper_z_active_mctruth',
       'standoff_active_mctruth', 'standoff_r_active_mctruth',
       'standoff_z_active_mctruth', 'm_nQ', 'm_nOPCal', 'm_nOPCollected',
       'm_DNNvalue', 'NESTBugFound', 'ratio', 'fGenX', 'fGenY', 'fGenZ',
       'Branch_ratio', 'Seed', 'Nevts']
      )